# Euler buckling of a column

In this exercise, we consider a **slender column subjected to axial compression** and study the phenomenon of **Euler buckling**.

For sufficiently small compressive loads, the column remains straight and deforms only axially. When the applied load reaches a critical value, the straight configuration loses stability and the column buckles laterally. This critical load is known as the **Euler buckling load**.

In the following, we will model a column using geometrically exact beam finite elements and investigate:
- the onset of buckling,
- the influence of boundary conditions on the critical load,
- and the post-buckling behavior of the column.

The figure below shows the undeformed configuration of the column together with the applied boundary conditions and loading.

<img src="doc/euler_column.png" alt="Euler column" width="400">

The beam has a length of $L = 0.3\text{m}$, a circular cross-section with radius $r = 0.01\text{m}$, a Young's modulus of $E = 2.1\cdot 10^{11}\text{N}/\text{m}^2$. A load $P$ is applied at the right end in negative $x$-direction.


In [ ]:
# We have to import the required packages, classes and functions.
# These imports will also be used in the exercises below, they only
# have to be included once.
import matplotlib.pyplot as plt
import numpy as np
from beamme.core.mesh import Mesh
from beamme.four_c.element_beam import Beam3rLine2Line2
from beamme.four_c.material import MaterialReissner

from utils.lecture_utils import (
    create_beam_mesh_line_2d,
    create_boundary_condition_2d,
    get_force_displacement_data,
    plot_beam_2d,
    run_four_c,
)

<div class="alert alert-info" role="alert">
  <strong>Exercise 3.1: Column under axial compression (pre-buckling)</strong>
  
  Add the correct boundary conditions to the following code and apply a load $P=100.000\text{N}$ to the right end of the column and simulate the model (the load is smaller than the critical load).
  Use 20 elements for the discretization and a single load step.
</div>

In [ ]:
E = 2.1e11
radius = 0.01
L = 0.3
P = 100000.0

mesh = Mesh()
material = MaterialReissner(radius=radius, youngs_modulus=E)

beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=20
)

# === BLOCK SOLUTION === # fmt: skip
# Displacement boundary conditions
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y"]
)
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="dirichlet", directions=["y"]
)

# Load at the end
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="neumann", directions=["x"], values=[-P]
)
# === BLOCK STUDENT === # fmt: skip
# TODO: Uncomment and complete the following code to define the boundary conditions
# Displacement boundary conditions at the left end
# create_boundary_condition_2d(
#     mesh, beam_set["start"], bc_type="...", directions=[...]
# )

# Displacement boundary conditions at the right end
# create_boundary_condition_2d(
#     mesh, beam_set["end"], bc_type="...", directions=[...]
# )

# Force boundary conditions at the left end
# create_boundary_condition_2d(
#     mesh, beam_set["end"], bc_type="...", directions=[...], values=[...]
# )
# === BLOCK END === # fmt: skip

n_steps = 1
run_four_c(mesh=mesh, simulation_name="ex_3_1", n_steps=n_steps, display_log=False)

plot_beam_2d("ex_3_1")

<div class="alert alert-info" role="alert">
  <strong>Exercise 3.2: Identification of the critical buckling load</strong>
  
  1. Compute the critical buckling load $P_\mathrm{cr}$
     $$
     P_\mathrm{cr} = \frac{\pi^2 E I}{L^2}.
     $$
     *Hint:* The moment of inertia for a circular cross-section is given as $I = \frac{\pi r^4}{4}$.
  1. Apply a load $P=5 P_\mathrm{cr}$ to the code from the previous exercise.
     (Use the suggested 20 elements for the discretization and 20 load steps)
  1. Does the column buckle? Explain your observation.
</div>

In [ ]:
E = 2.1e11
radius = 0.01
L = 0.3

# === BLOCK SOLUTION === # fmt: skip
moment_of_inertia = np.pi * radius**4 / 4
critical_load = np.pi**2 * E * moment_of_inertia / (L**2)
# === BLOCK STUDENT === # fmt: skip
# TODO: Compute the critical buckling load here
# critical_load = ...
# === BLOCK END === # fmt: skip
print("Critical buckling load:\n", critical_load)

mesh = Mesh()
material = MaterialReissner(radius=radius, youngs_modulus=E)

beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=20
)

# Displacement boundary conditions
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y"]
)
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="dirichlet", directions=["y"]
)

# Load at the end
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["x"],
    values=[-(5 * critical_load)],
)

n_steps = 20
run_four_c(mesh=mesh, simulation_name="ex_3_2", n_steps=n_steps)

plot_beam_2d("ex_3_2")

<!-- === BLOCK SOLUTION === -->
<div class="alert alert-success" role="alert">
  3. The column does not buckle, even though the applied load is larger than the critical load. This is due to the fact that in a numerical simulation, perfect geometry and loading conditions are assumed. Therefore, no imperfections exist that could trigger the buckling behavior. In reality, small imperfections always exist, which lead to buckling when the critical load is reached.
</div>
<!-- === BLOCK STUDENT === -->
<!-- === BLOCK END === -->

<div class="alert alert-info" role="alert">
  <strong>Exercise 3.3: Post-buckling behavior with initial imperfection</strong>

  In the previous exercise, the critical buckling load of a perfectly straight column was identified. However, we could observe that numerically, we can not trigger buckling for the perfectly straight column. In practice it is common to introduce small initial imperfections to the model to trigger buckling. In this case we will do so by applying a small imperfection moment at the right boundary condition, as shown in the following figure.

  <img src="doc/euler_column_with_moment.png" alt="Euler column with initial imperfection" width="400">
  
  This is not only relevant for numerical purposes, but also for engineering practice. Real structures always exhibit small geometric imperfections, which can strongly influence the buckling and post-buckling behavior.
  Thus is important to understand how imperfections affect the structural response.

  1. Adapt the code such that initial imperfection moment is applied at the right boundary.

     *Hint:* Use the function `create_boundary_condition_2d` accepts the flag `linear_increase=False` which instantly applies the full moment at the beginning of the simulation, instead of linearly increasing it over the load steps. This is desired for the imperfection moment.
  1. Run the simulation with a load $P=5 P_\mathrm{cr}$. Use an imperfection moment of $M_\text{imp} = 0.01 \cdot P_{\mathrm{cr}} \cdot L$ (use 20 elements for the discretization and 20 load steps). Inspect the results. How does the imperfection influence the buckling behavior?
</div>

In [ ]:
E = 2.1e11
radius = 0.01
L = 0.3
n_steps = 20

moment_of_inertia = np.pi * radius**4 / 4
critical_load = np.pi**2 * E * moment_of_inertia / (L**2)

imperfection_moment = critical_load * L * 0.01

mesh = Mesh()
material = MaterialReissner(radius=radius, youngs_modulus=E)

beam_set = create_beam_mesh_line_2d(
    mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=20
)

# Displacement boundary conditions
create_boundary_condition_2d(
    mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y"]
)
create_boundary_condition_2d(
    mesh, beam_set["end"], bc_type="dirichlet", directions=["y"]
)

# === BLOCK SOLUTION === # fmt: skip
# Apply imperfection moments at the right end
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["theta"],
    values=[-imperfection_moment],
    linear_increase=False,
)
# === BLOCK STUDENT === # fmt: skip
# TODO: Uncomment and complete the following code to apply the imperfection moment at the right end
# create_boundary_condition_2d(
#     mesh,
#     beam_set["end"],
#     bc_type="...",
#     directions=["..."],
#     values=[...],
#     linear_increase=False,
# )
# === BLOCK END === # fmt: skip

# Load at the end
create_boundary_condition_2d(
    mesh,
    beam_set["end"],
    bc_type="neumann",
    directions=["x"],
    values=[-(5 * critical_load)],
)

run_four_c(mesh=mesh, simulation_name="ex_3_3", n_steps=n_steps)

plot_beam_2d("ex_3_3")

<!-- === BLOCK SOLUTION === -->
<div class="alert alert-success" role="alert">
  2. The imperfection triggers the buckling behavior and the column buckles under the applied load (which is larger than the critical load).
</div>
<!-- === BLOCK STUDENT === -->
<!-- === BLOCK END === -->

<div class="alert alert-info" role="alert">
  <strong>Exercise 3.4: Force displacement curve</strong>

  Consider the column model from the previous exercises. A small initial imperfection is introduced by applying equal and opposite bending moments at the ends of the column.

  For the following examples, use $P=2 P_\mathrm{cr}$, 20 elements for the discretization and 50 load steps.

  1. Perform simulations for different imperfection magnitudes, e.g., $M_{\text{imp}} \in \{0.001, 0.002, 0.005,0.01\}\cdot P_{\mathrm{cr}} \cdot L$.
  1. Plot the force $P$ versus lateral displacement $v$ in the middle of the beam.
  1. Indicate the analytical Euler buckling load in the plot.
  1. Discuss the following aspects based on your results:
     1.  How does the imperfection magnitude influence the buckling behavior?
     2. What kind of post-buckling behavior can be observed?
     3. Even with small imperfections, the column does not buckle exactly at the critical load. Why is that?

  *Hint:* The function `get_force_v_arrays` runs a simulation for a given imperfection moment and returns the force and lateral displacement arrays for plotting.
</div>

In [ ]:
def get_force_v_arrays(imperfection_moment_factor):
    """Run a simulation for a given imperfection moment and return the force-displacement data."""

    radius = 0.01
    L = 0.3
    n_steps = 50

    moment_of_inertia = np.pi * radius**4 / 4
    critical_load = np.pi**2 * E * moment_of_inertia / (L**2)

    imperfection_moment = critical_load * L * imperfection_moment_factor

    mesh = Mesh()
    material = MaterialReissner(radius=radius, youngs_modulus=E)

    beam_set = create_beam_mesh_line_2d(
        mesh, Beam3rLine2Line2, material, [0, 0], [L, 0], n_el=20
    )

    create_boundary_condition_2d(
        mesh, beam_set["start"], bc_type="dirichlet", directions=["x", "y"]
    )
    create_boundary_condition_2d(
        mesh, beam_set["end"], bc_type="dirichlet", directions=["y"]
    )
    create_boundary_condition_2d(
        mesh,
        beam_set["end"],
        bc_type="neumann",
        directions=["theta"],
        values=[-imperfection_moment],
        linear_increase=False,
    )
    create_boundary_condition_2d(
        mesh,
        beam_set["end"],
        bc_type="neumann",
        directions=["x"],
        values=[-(2 * critical_load)],
    )

    run_four_c(mesh=mesh, simulation_name="ex_3_4", n_steps=n_steps, display_log=False)
    print(f"Finished simulation for imperfection moment: {imperfection_moment}")

    force, displacement = get_force_displacement_data("ex_3_4", [0.5 * L, 0])
    v = displacement[:, 1]
    return force, v


# === BLOCK SOLUTION === # fmt: skip
for imperfection_moment_factor in [0.001, 0.002, 0.005, 0.01]:
    force, v = get_force_v_arrays(imperfection_moment_factor)
    plt.plot(
        v,
        force,
        label=f"$M_{{imp}} = {imperfection_moment_factor} \\cdot P_{{cr}} \\cdot L$",
    )

plt.axhline(y=critical_load, color="k", linestyle="dashed", label="Critical Load")
plt.legend()
plt.xlabel("$v$")
plt.ylabel("$P$")
plt.title("Force displacement curves for different imperfection moments")
plt.grid(True)
plt.show()
# === BLOCK END === # fmt: skip

<!-- === BLOCK SOLUTION === -->
<div class="alert alert-success" role="alert">

  4.1 With increasing imperfection, the column starts to buckle earlier and the force displacement curve becomes smoother.
  
  4.2 The post-buckling behavior is stable. This means that the column does not collapse with if the load eclipses the critical value. An instable post-buckling behavior would be characterized by a negative slope in the force-displacement curve.

  4.3 The reason for that is the fact that the analytical Euler buckling load is derived under the assumption of shear stiff beams. The employed beam theory is shear deformable, which results in a "softer" structure that buckles at slightly smaller loads.
</div>
<!-- === BLOCK STUDENT === -->
<!-- === BLOCK END === -->